In [1]:
import numpy
import pandas

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

import pythainlp

from sklearn.cluster import KMeans

import re
import math
import sys

from data_tokenizer import load_corpus

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TescoLotus.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 269
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

appear_rate = 1
dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = []
for word in dictionary.keys():
    if len(dictionary[word]) <= 1:
        letter_words.append(word)
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1313 words
filter frequent words: 540 words
filter letter words: 539 words
filter stop words: 352 words


In [4]:
# bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
    print(temp_corpus[-1])
idx_corpus = temp_corpus

['พนักงาน', 'พนักงาน', 'โลตัส', 'โลตัส', 'ทำ', 'เอ็กเพลส', 'พนักงาน', 'คน', 'ทำ', 'ขาย', 'คน', 'คน', 'เวลา', 'กิน', 'ข้าว', 'ผม', 'คน', 'พนักงาน', 'เหมือน', 'พนักงาน', 'คน', 'ผม', 'รู้', 'พนักงาน', 'ตั้งใจ', 'ทำ', 'งาน', 'คน']
['เลิก', 'ข้าว', 'ดี', 'อี', 'เครื่อง', 'น้ำ']
['ถุง', 'งง', 'ถุง', 'กก', 'เจอ', 'น้ำยา', 'ซัก', 'ผ้า', 'น้ำ', 'ยา', 'ผ้า', 'นุ่ม']
['ถุง', 'แถม', 'ลด', 'ลอง', 'ดู', 'ถุง', 'หนา']
['ทำ', 'เดือน', 'พนักงาน', 'คน', 'คน', 'คน', 'นึง', 'พัน', 'เช้า', 'เย็น']
['อย่า', 'บังคับ', 'น้อง', 'เชียร์', 'อย่า', 'งี่', 'เง่า', 'กี่', 'ปี']
['โลตัส', 'โลตัส', 'เอกเพรส', 'แม่ง', 'เกลียด', 'พนักงาน']
['คน', 'คน', 'คน', 'ดี', 'คน', 'ดี', 'ผม', 'โลก', 'ผม', 'ทำ', 'งาน', 'บริการ', 'เหมือน', 'ยืน', 'จุด', 'ยืน', 'รู้', 'พนักงาน', 'เหมือน', 'ปรับปรุง', 'ดี', 'ปรับปรุง', 'ตัว', 'ดี', 'ดี']
['พนักงาน', 'ทำ', 'หน้า', 'ตูด', 'คน', 'เหมือน', 'บังคับ', 'ทำ', 'งาน', 'งั้น']
['เอ็กเพรส', 'อบรม', 'พนักงาน', 'ป่าว', 'อารม', 'ถาม', 'ราคา', 'ทำ', 'หน้า', 'พอใจ', 'ติด', 'ราคา', 'ถาม', 'หรอก', 'ราค

In [5]:
unique_words = [dictionary[id] for id in range(len(dictionary))]

array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)
for i, doc in enumerate(idx_corpus):
    for word in doc:
        array[i, dictionary.token2id[word]] += 1
    
    ## normalization
    if len(doc) != 0:
        array[i] = numpy.divide(array[i], len(doc))

bow_corpus = pandas.DataFrame(array, columns=unique_words, dtype=float)
bow_corpus.head()

,กิน,ขาย,ข้าว,คน,งาน,ตั้งใจ,ทำ,ผม,พนักงาน,รู้,...,ฉีด,ล้อ,สะอาด,ร้อน,แจก,เขียว,พลังงาน,แอร์ไม่,สลิป,นวมินทร์
0,0.035714,0.035714,0.035714,0.214286,0.035714,0.035714,0.107143,0.071429,0.214286,0.035714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.300000,0.000000,0.000000,0.100000,0.000000,0.100000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### K-Mean

In [22]:
num_clusters = 6
kmeans = KMeans(n_clusters=num_clusters).fit(bow_corpus)
predicted_labels = kmeans.labels_

In [7]:
result = pandas.DataFrame()
result['comment'] = corpus
result['tokenized_comment'] = idx_corpus
result['label'] = labels
result['predicted_label'] = predicted_labels
result.head()

,comment,tokenized_comment,label,predicted_label
0,บางครั้งพนักงานโลตัสเอ็กเพลส กับพนักงานโลตัสให...,"[พนักงาน, พนักงาน, โลตัส, โลตัส, ทำ, เอ็กเพลส,...",,1
1,เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพีย...,"[เลิก, ข้าว, ดี, อี, เครื่อง, น้ำ]",,1
2,ถุงงง ถุงมึงบอบบางมากกก ยิ่งเจอน้ำยาซักผ้า น้...,"[ถุง, งง, ถุง, กก, เจอ, น้ำยา, ซัก, ผ้า, น้ำ, ...",,1
3,เราไม่เคยรับถุงเลย แถมได้แต้มกรีนพ้อยไว้ใช้เป็...,"[ถุง, แถม, ลด, ลอง, ดู, ถุง, หนา]",,5
4,เคยไปทำ เอ้กซ์เพรส เดือนเดียว พอเลย พนักงานต่...,"[ทำ, เดือน, พนักงาน, คน, คน, คน, นึง, พัน, เช้...",,1


### Dimension Reduction

In [11]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)
average_doc_size

word_freq = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, word_freq[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

In [12]:
dictionary = Dictionary(filtered_corpus)

# dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))
# print(len(dictionary))

unique_words = [dictionary[id] for id in range(len(dictionary))]

array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)
for i, doc in enumerate(filtered_corpus):
    for word in doc:
        array[i, dictionary.token2id[word]] += 1

    ## normalization
#     if len(doc) != 0:
#         array[i] = numpy.divide(array[i], len(doc))
        
bow_corpus = pandas.DataFrame(array, columns=unique_words, dtype=float)
bow_corpus.head()

,ทำ,พนักงาน,ข้าว,ดี,น้ำ,อี,เครื่อง,เลิก,กก,งง,...,พลังงาน,คอย,แอร์ไม่,สลิป,คู,ปอง,นวมินทร์,สิบ,ห้า,แจก
0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
num_clusters = 6
kmeans = KMeans(n_clusters=num_clusters).fit(bow_corpus)
predicted_labels = kmeans.labels_

In [24]:
result = pandas.DataFrame()
result['comment'] = corpus
result['tokenized_comment'] = filtered_corpus
result['label'] = labels
result['predicted_label'] = predicted_labels
result.head()

,comment,tokenized_comment,label,predicted_label
0,บางครั้งพนักงานโลตัสเอ็กเพลส กับพนักงานโลตัสให...,"[พนักงาน, พนักงาน, พนักงาน, พนักงาน, พนักงาน, ...",,2
1,เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพีย...,"[ดี, เลิก, น้ำ, เครื่อง, ข้าว, อี]",,4
2,ถุงงง ถุงมึงบอบบางมากกก ยิ่งเจอน้ำยาซักผ้า น้...,"[ถุง, ถุง, เจอ, น้ำ, งง, น้ำยา, ผ้า, ผ้า, กก]",,0
3,เราไม่เคยรับถุงเลย แถมได้แต้มกรีนพ้อยไว้ใช้เป็...,"[ถุง, ถุง, ลด, แถม, ดู, หนา, ลอง]",,0
4,เคยไปทำ เอ้กซ์เพรส เดือนเดียว พอเลย พนักงานต่...,"[พนักงาน, ทำ, คน, คน, คน, นึง, เดือน, เย็น, เช้า]",,3


### Result

In [15]:
label_count = numpy.unique(predicted_labels, return_counts=True) 
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([0, 1, 2, 3, 4, 5], dtype=int32), array([ 63,   4,  21,  33,  23, 125]))
		percent
0  |	63	1.0


In [46]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [119]:
token_widget = widgets.ToggleButtons(
    options=[num for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_token_widget_click(change):
    clear_output()
    display(token_widget)
    for index, value in result[result['predicted_label'] == change['new']]['tokenized_comment'].iteritems():
        print(index, value)

result_widget.observe(on_token_widget_click, names='index')
on_token_widget_click({'new' : 1})

ToggleButtons(options=(0, 1, 2, 3, 4, 5), value=0)

21 ['พนักงาน', 'ลูกค้า', 'ลูกค้า', 'หน้า', 'คน', 'ถุง', 'ซื้อ', 'ใส่', 'ใส่']
73 ['พนักงาน', 'พนักงาน', 'ลูกค้า', 'ลูกค้า', 'หน้า', 'หน้า', 'หน้า', 'โลตัส', 'บริการ']
96 ['พนักงาน', 'พนักงาน', 'พนักงาน', 'ลูกค้า', 'หน้า', 'ทำ', 'โลตัส', 'โลตัส', 'คน']
105 ['ลูกค้า', 'หน้า', 'หน้า', 'หน้า', 'ซื้อ', 'ใส่', 'รอ', 'พนง', 'ชอบ']
149 ['พนักงาน', 'หน้า', 'หน้า', 'คน', 'ใส่', 'ใส่', 'ดี', 'พูดจา', 'อบรม']


In [116]:
comment_widget = widgets.ToggleButtons(
    options=[num for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_comment_widget_click(change):
    clear_output()
    display(comment_widget)
    for index, value in result[result['predicted_label'] == change['new']]['comment'].iteritems():
        print(index, value)

result_widget.observe(on_comment_widget_click, names='index')
on_comment_widget_click({'new' : 0})

ToggleButtons(options=(0, 1, 2, 3, 4, 5), value=0)

2 ถุงงง  ถุงมึงบอบบางมากกก ยิ่งเจอน้ำยาซักผ้า น้ำยาปรับผ้านุ่มนะ พร้อมขาดดด
3 เราไม่เคยรับถุงเลย แถมได้แต้มกรีนพ้อยไว้ใช้เป็นส่วนลดได้ด้วย ลองดู เตรียมถุงหนาๆไปเองใช้ได้หลายครั้ง
11 เลือกผัก เลือกแล้ว เลือกอีก พอมาคิดเงินพนักงานยัดผักใส่ถุงจนผักหัก โอ้ยใจ ใจสลาย
15 ถุงบางมากนี่เรื่องจริง ลูกค้าบางคนเค้าไม่ได้มีรถส่วนตัว หิ้วขึ้นสองแถว ขึ้นรถเมล์ ก็ต้องเข้าใจเค้าด้วย ขอถุงซ้อนเพิ่มก็กลายเป็นเพิ่มการใช้ถุงพลาสติกเข้าไปอีก ก็ทำให้มันหนากว่านี้หน่อยแล้วใช้ใบเดียวไม่ดีกว่าเหรออย่างโลตัสเอกเพรส ส่วนใหญ่คืออยู่ในเขตหมู่บ้าน ลูกค้าเดินมาซื้อ ขี่จักรยานมาซื้อ ถ้าไม่ขอให้ซ้อนถุงนี่หิ้วกลับไม่ถึงบ้านค่ะ บางทีเลยต้องเข้า 711 ถ้าซื้อของที่มีเหมือนกัน
23 ควย ค่ะ พนักงานแม่งเกือบทุกคนเลย ไม่มีความคิด วิเคราะห์ แยกแยะไม่ได้ว่าเวลาลูกค้าซื้อผัก ซื้อหมูบด น้ำยาล้างจาน น้ำปลา หรืออะไรก็แล้วแต่ ถุงมึงมีใบใหญ่สุดแค่ไหนมึงเอามาใส่แม่งรวมกันหมดทุกอย่าง ผักกูเลือกดีๆเอาไม่ช้ำ อีดอกเอาไว้ล่างสุด เอาแพ็คหมูบดทับ
49 ถุงใส่สินค้าบางมากควรปรับปรุงได้แล้วนะค่ะทั้งและสาขาใหญ่ทั้งหมด
56 1 งงใบเสร็จครับ อะไรแถม อะไรลด